# Нейросети в задачих обработки текстов

Основано на коде из курса [Глубинное обучение ФКН](https://github.com/aosokin/dl_cshse_ami).

**Разработчик: Алексей Озерин, Ирина Сапарина**

# Генерация коротких текстов с помощью RNN и Transformer


Генерировать тексты можно как с помощью RNN, так и с помощью Transformer, предсказывая следующий символ последовательности по предыдущим. Мы будем использовать архитектуру Transformer.

В этом задании предлагается написать и проучить на небольшом датасете имен генеративную модель на основе символов.

In [ ]:
%matplotlib inline

import numpy as np
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

from IPython.display import clear_output


random.seed(2021)
np.random.seed(2021)
torch.manual_seed(2021)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(2021)

В файле `names` находится ~8k имен на латинице.

Модель будет получать на вход имя `Amandy` и выдавать его же, только со сдвигом: `mandy `.

Чтобы сеть училась генерировать заглавные буквы, добавим в начало специальный токен `_`.

Также нам потребуется правило для останова генерации (это может быть просто ограничение на количество шагов). С другой стороны, можно добавить в конец каждого примера обучающей выборки специальный `<EOS>` токен. В данном случае обозначим его `#`:

```
_Amandy --> Amandy#
```

Можно прекращать генерацию при досрочном выпадании `<EOS>`.

Для генерации на каждом шаге будем подавать на вход букву, предсказанную на предыдущем.



In [ ]:
import os
start_token = "_"
eos = '#'

with open("names") as f:
    names = f.readlines()
    names = [start_token + name.strip() + eos for name in names]

names = list(set(names))  # в датасете есть повторы
print('There are {} names: '.format(len(names)))
for x in names[::1000]:
    print(x)

# Подготовка и знакомство с данными
**(0.1 балла)**

In [ ]:
# TODO: постройте частоты употреблений букв
<your code>
# HINT: для графика возьмите plt.bar

In [ ]:
# в датасете есть слова с разными длинами
MAX_LENGTH = max(map(len,names))
print("max length =", MAX_LENGTH)

plt.title('Sequence length distribution')
plt.hist(list(map(len,names)), bins=25);

In [ ]:
names[:10]

In [ ]:
# TODO: отберите уникальные токены и заполните два словаря для конвертации токенов <-> индексы
# сделайте так, чтобы pad_token имел номер 0
    
tokens = <your code>
    
tok2id = <your code>
id2tok = <your code>

n_tokens = len(tokens)
print ('There are {} tokens',n_tokens)

assert 50 < n_tokens < 60

print('Vocabular: ' + "".join(tokens))

## Работа с последовательностями произвольной длины в pytorch

Нам нужно уметь генерировать батчи тензоров `[bs, 1, seq_len]`.
Но в нашем датасете семплы разной длины:

- мы могли бы подрезать все до минимальной
- паддить до максимальной
- выбрать какую-то среднюю длину

**(0.1 балла)** Разбейте датасет на train и validate:

In [ ]:
# сделаем датасет выдающий закодированные имена:
class NamesDataset(Dataset):
    def __init__(self, names):
        self.names = names
    
    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, item):
        entry = self.names[item]
        return dict(
            encoded=entry,
        )

encoded = []
for entry in tqdm(names):
    encoded.append(tok2id(entry))

<your code>
trainset = NamesDataset(...)
valset = NamesDataset(...)

Давайте соберем наивный DataLoader и посмотрим как он делает батчи:


In [ ]:
trainloader = DataLoader(trainset, batch_size=8, shuffle=True)
it = iter(trainloader)

In [ ]:
batch = next(it)['encoded']
batch

В моем случае, результат запуска был таков:
```
[tensor([1, 1, 1, 1, 1, 1, 1, 1]),
 tensor([ 6,  7,  6, 15,  5,  6,  5, 62]),
 tensor([ 48,  34,  83,   7,  32, 221,  22,  43]),
 tensor([  5, 143,  37,  36, 129,  12,  11,  66]),
 tensor([  73, 1258,  279,    8,    6,  555,   41,   10]),
 tensor([  8, 140,   8, 628,  20,  96,  13, 270]),
 tensor([  47,    4,   15,   18,   55,  269,    6, 1287]),
 tensor([ 58,   2,  13, 140, 193, 140, 171, 140])]
```

Какие странности здесь видны?
1. Это не тензор, а список тензоров. Соответственно при итерировании по нулевой размерности (`batch[i, :]`) мы будем получать не i-пример, а i-токены для всех примеров в батче. Это не проблема, но отличается от ожидаемого поведения.
2. На `<EOS>` (2) оканчивается только один пример, остальные подрезаны под его длину. И вот это уже проблема.

Мы бы хотели западдить все примеры до длины максимального в батче. 
Но на этапе подготовки примера (в функции `__getitem__`) мы не знаем соседей по батчу!
Для того чтобы поменять логику склейки батчей нам понадобиться написать свою функцию `collate_fn` в конструкторе DataLoader:

```
def collate_fn(samples):
    # samples -- список семплов-словарей
    <...>
    return batch
```

**(0.1 балл)** Напишите функцию `collate_fn`, которая _правильно_ паддит names-последовательности и объединяет их в батчи, где `batch[i, :]` выдает токены для `i`-примера.

Ожидаемый выход (для последовательности с левым паддингом):

```
tensor([[   1,   10, 3429,  405,  113,  676,   10, 1031,  140,    4,    2],
        [   0,    1,   57,   18,   23,   19,   61,    7,  140,    4,    2],
        [   0,    0,    0,    1,   16,   17, 1131,  416,  140,    4,    2],
        [   0,    0,    0,    1,   13,  465,   75,  197,  140,    4,    2],
        [   0,    0,    0,    1,    6,  302,   13,  144,  140,    4,    2],
        [   0,    1,    6,   59,  205,  167,    8,   15,  140,    4,    2],
        [   0,    0,    0,    0,    1,    6,   14,  678,  140,    4,    2],
        [   0,    0,    1,    5,   29,   67,    6,   14,  140,    4,    2]])
```

In [ ]:
def collate_fn(samples):
    # <your code>
    return dict(
        encoded=...
    )
    

trainloader = DataLoader(trainset, batch_size=8, shuffle=True, collate_fn=collate_fn)
it = iter(trainloader)
next(it)['encoded']

# Char-RNN для имен (0.2 балла)

Вам нужно написать сеть, кодирующую номера входных символов с помощью таблицы Embeddings. 
Получившиеся тензоры пропустить через RNN ячейку, затем преобразовать в логиты для предсказания номера нового символа.

In [ ]:
# NB: обратите внимание на порядок осей при вызове forward
# http://pytorch.org/docs/master/nn.html#recurrent-layers

# Сделайте возможность выбора типа ячейки, RNN, GRU или LSTM
# TODO: заполните пропуски. Функция forward будет вызываться на каждый шаг нами

class NameRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, output_size, cell="lstm", n_layers=1):
        super(NameRNN, self).__init__()
        # добавьте возможность выбрать тип ячейки RNN/LSTM
        
        
    def forward(self, input, hidden):
        <...>

    def init_hidden(self, batch_size):
        <...>

# Натренируйте модель (0.2 балла)

Возьмите трейнер с предыдущих занятий, натренируйте модель

# Генерация по argmax (0.2 балла)

In [ ]:
# Напишите функцию генерации продолжения строки
def pick_by_argmax(logits):
    <your code>

def ids2string(ids):
    return "".join(id2tok[_] for _ in ids)


def gen_continuation(model, prefix="_"):
       # TODO: сначала подайте на вход префикс
    # нас интересует последний output, чтобы получить первое предсказание
    <your code>
    
    # TODO: затем сгенерируйте несколько последующих символов
    # outs -- это массив с номерами токенов
    <your code>
    
    print(prefix + '|'+ ids2string(outs))
    
gen_continuation(model, " Ku")

# Генерация с семплированием (0.2 балла)

Обычный софтмакс 
$$p_i = \frac{\exp (x_i)}{\sum \exp (x_j)}$$
можно модернизировать с помощью температуры:
$$p_i = \frac{\exp (x_i / T)}{\sum \exp (x_j / T)}$$

Это позволит плавно переходить от выбора наиболее вероятного элемента ($T << 1$) до практически равновероятного ($T >> 1$)


In [ ]:
# Напишите функцию генерации батчами с семплированием из распределения и температурой
def batch2string(ids, prefix):
    # модифицируйте ids2string для работы с батчами
    <your code>

def pick_by_distribution(logits):
    # превратите логиты в распределение
    # затем семлируйте из него batch примеров
    <your code>


def gen_continuation_temp(model, prefix="_", temperature=1.0, n=10):
    # аналогично, сначала подайте на вход префикс
    # нас интересует последний output, чтобы получить первое предсказание
    <your code>
    
    # затем, сгенерируйте n последующих символов
    # в outs положите матрицу номеров токенов и отобразите ее
    
    print(batch2string(outs, prefix + '|'))
    
gen_continuation_temp(model, prefix=" An", temperature=0.5, n=10)

# Char-Transformer для имен (1.0 дополнительные баллы)

Вам нужно написать сеть, кодирующую входные символы и их позиции с помощью таблиц Embeddings. 
Получившиеся тензоры пропустить через `TransformerEncoder`, затем преобразовать в логиты для предсказания новых символов.

Transformer может обрабатывать сразу всю последовательность за один проход. Для того, чтобы у модели не было возможности "заглянуть в будущее", то есть использовать информацию о впреди идущих символах, необходимо сгенерировать маску. `TransformerEncoder` должен принимать на вход последовательность символов и маску.    
![Transformer](https://drive.google.com/uc?export=view&id=1gXILzT3mGgc0mGlvqY-6R4bGs3Lx2YxM)
Картинка из [illustrated transformer](http://jalammar.github.io/illustrated-transformer/)

In [ ]:
# TODO: заполните пропуски

class NameTransformer(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, n_layers=2, n_head=2, dropout=0.1):
        super(NameTransformer, self).__init__()
        self.vocab_size = vocab_size

        <your code>

    def _generate_square_subsequent_mask(self, seq_len):
        # TODO: сгенерируйте маску размера seq_len x seq_len
        # если во время кодирования i-го символа j-й символ доступен, 
        # то (i,j) элемент маски равен 0, иначе -inf
        
        <your code>

        return mask
        
    def forward(self, input):

        <your code>

        return output

# Натренируйте модель

И убедитесь, что она работает адекватно